In [6]:
import os, json
import pandas as pd
import re
from pythainlp.util import thai_strptime
from datetime import datetime
import copy

with open("data/provinces.json", 'r', encoding='utf-8') as file:
    data = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

for p in data:
    p["events"] = 0 

with open("data/analysis.json", 'r', encoding='utf-8') as file:
    analysis = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

data_detail = []
# SCORE EVENT
for a in analysis:
    ps = a["province"].split(" | ")
    for p in ps:
        for d in data:
            if d["province"] == p:
                new_d = copy.deepcopy(d)
                new_d["ISO8601"] = a["ISO8601"]
                new_d.pop("latitude", None)
                new_d.pop("longitude", None)
                new_d.pop("events", None)
                new_d["id"] = a["id"]
                new_d["title"] = a["title"]
                data_detail.append(new_d)
                d["events"] += 1
        
# FILTER EVENTS ZERO
data = [x for x in data if x["events"] > 0]

# โหลดข้อมูลเป็น DataFrame
df = pd.DataFrame(data)
df2 = pd.DataFrame(data_detail)

print(df)

import folium
from folium.plugins import HeatMap

# สร้างแผนที่เริ่มต้น
m = folium.Map(location=[13.7563, 100.5018], zoom_start=6)

# เตรียมข้อมูลสำหรับ Heatmap
heat_data = [[row["latitude"], row["longitude"], row["events"]] for index, row in df.iterrows()]

# เพิ่ม Heatmap ลงในแผนที่
HeatMap(heat_data, max_zoom=12).add_to(m)

# แสดงแผนที่
m.save("thailand_heatmap.html")
print("Heatmap saved as 'thailand_heatmap.html'")

filename = "data/heatmap.xlsx"
df.to_excel(filename, index=False)  # บันทึกเป็น Excel

filename = "data/analysis-detail.xlsx"
df2.to_excel(filename, index=False)  # บันทึกเป็น Excel

         province   latitude   longitude  events
0   กรุงเทพมหานคร  13.756539  100.555091       8
1          กระบี่   8.014196   98.933196       1
2       กาฬสินธุ์  16.589940  103.613395       2
3         ขอนแก่น  16.354786  102.597114       4
4        จันทบุรี  12.683440  102.114321       1
..            ...        ...         ...     ...
64        อ่างทอง  14.613110  100.382438      20
65       อุดรธานี  17.407981  102.887755       6
66      อุตรดิตถ์  17.639478  100.327030       2
67      อุทัยธานี  15.371043   99.792609      11
68    อุบลราชธานี  15.274664  105.023414       6

[69 rows x 4 columns]
Heatmap saved as 'thailand_heatmap.html'
